
## Manhattan Grocery Store Location Suggestion

Download and import all the dependencies that will be needed.


<a id='item1'></a>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.

In [2]:
from bs4 import BeautifulSoup
import requests
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.

In [4]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [5]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [8]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In [12]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


#### Create a map of New York with neighborhoods superimposed on top.

In [13]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [14]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [15]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [16]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [17]:
CLIENT_ID = 'MKU3AYOKPREAGHVUW0WOUH0TBVQ0G14JB5IG3OH4F15QNCJU' # your Foursquare ID
CLIENT_SECRET = 'YKPOAXZS3Y5LRLGAR2TFEBAA4ZGYATOLZI15CKIKFXLQJP3M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MKU3AYOKPREAGHVUW0WOUH0TBVQ0G14JB5IG3OH4F15QNCJU
CLIENT_SECRET:YKPOAXZS3Y5LRLGAR2TFEBAA4ZGYATOLZI15CKIKFXLQJP3M


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [18]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

Get the neighborhood's latitude and longitude values.

In [19]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [20]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL




'https://api.foursquare.com/v2/venues/explore?&client_id=MKU3AYOKPREAGHVUW0WOUH0TBVQ0G14JB5IG3OH4F15QNCJU&client_secret=YKPOAXZS3Y5LRLGAR2TFEBAA4ZGYATOLZI15CKIKFXLQJP3M&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

Double-click __here__ for the solution.
<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 

Send the GET request and examine the resutls

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bf21b23351e3d0bc86654cc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin' Donuts,Donut Shop,40.876993,-73.906507


And how many venues were returned by Foursquare?

In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [26]:
# type your answer here
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )




Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [27]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3289, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin' Donuts,40.876993,-73.906507,Donut Shop


Let's check how many venues were returned for each neighborhood

In [28]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,98,98,98,98,98,98
Carnegie Hill,100,100,100,100,100,100
Central Harlem,43,43,43,43,43,43
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,43,43,43,43,43,43
East Village,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [29]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 329 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [30]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant

In [31]:
restaurant_list = ['Pizza Place',
 'Diner',
 'Coffee Shop',
 'Donut Shop',
 'Seafood Restaurant',
 'Comfort Food Restaurant',
 'Steakhouse',
 'Sandwich Place',
 'Deli / Bodega',
 'Chinese Restaurant',
 'Cocktail Bar',
 'Greek Restaurant',
 'English Restaurant',
 'Noodle House',
 'American Restaurant',
 'Bubble Tea Shop',
 'Bar',
 'New American Restaurant',
 'Malay Restaurant',
 'Vietnamese Restaurant',
 'Hotpot Restaurant',
 'Dim Sum Restaurant',
 'Italian Restaurant',
 'Spanish Restaurant',
 'Asian Restaurant',
 'Snack Place',
 'Restaurant',
 'Dumpling Restaurant',
 'Bakery',
 'Taiwanese Restaurant',
 'Dessert Shop',
 'Shanghai Restaurant',
 'Austrian Restaurant',
 'Mexican Restaurant',
 'Ramen Restaurant',
 'Café',
 'Burger Joint',
 'Tapas Restaurant',
 'Breakfast Spot',
 'Wine Shop',
 'Caribbean Restaurant',
 'Indian Restaurant',
 'Latin American Restaurant',
 'Sushi Restaurant',
 'Wine Bar',
 'Beer Bar',
 'Arepa Restaurant',
 'Bistro',
 'Fast Food Restaurant',
 'Juice Bar',
 'Japanese Restaurant',
 'Mediterranean Restaurant',
 'Pub',
 'Gastropub',
 'BBQ Joint',
 'Japanese Curry Restaurant',
 'Beer Garden',
 'Falafel Restaurant',
 'Food & Drink Shop',
 'Cuban Restaurant',
 'Food Court',
 'French Restaurant',
 'Ethiopian Restaurant',
 'African Restaurant',
 'Bagel Shop',
 'Fried Chicken Joint',
 'Cafeteria',
 'Southern / Soul Food Restaurant',
 'Thai Restaurant',
 'Taco Place',
 'Vegetarian / Vegan Restaurant',
 'Burrito Place',
 'Hotel Bar',
 'Salad Place',
 'Turkish Restaurant',
 'German Restaurant',
 'Hot Dog Joint',
 'Peruvian Restaurant',
 'Sports Bar',
 'Afghan Restaurant',
 'Czech Restaurant',
 'Middle Eastern Restaurant',
 'Australian Restaurant',
 'South Indian Restaurant',
 'Caucasian Restaurant',
 'Szechuan Restaurant',
 'Hawaiian Restaurant',
 'Brazilian Restaurant',
 'Jewish Restaurant',
 'Scandinavian Restaurant',
 'Paella Restaurant',
 'Israeli Restaurant',
 'Udon Restaurant',
 'Eastern European Restaurant',
 'Gaming Cafe',
 'Lebanese Restaurant',
 'Moroccan Restaurant',
 'Korean Restaurant',
 'Pet Café',
 'Swiss Restaurant',
 'Soup Place',
 'Filipino Restaurant',
 'Argentinian Restaurant',
 'Whisky Bar',
 'Modern European Restaurant',
 'Karaoke Bar',
 'Wings Joint',
 'Irish Pub',
 'Belgian Restaurant',
 'Tiki Bar',
 'Kosher Restaurant',
 'Tibetan Restaurant',
 'Sake Bar',
 'Venezuelan Restaurant',
 'Molecular Gastronomy Restaurant',
 'Street Food Gathering',
 'Persian Restaurant',
 'Cambodian Restaurant',
 'Soba Restaurant',
 'Russian Restaurant',
 'Portuguese Restaurant',
 'Pakistani Restaurant']

grocery_list = ['Organic Grocery',
 'Grocery Store',
 'Market',
 'Frozen Yogurt Shop',
 'Supermarket',
 'Farmers Market',
 'Convenience Store',
 'Health Food Store',
 'Fish Market',
 'Flea Market']

needed_features = ['Neighborhood',
 'Pizza Place',
 'Diner',
 'Coffee Shop',
 'Donut Shop',
 'Seafood Restaurant',
 'Comfort Food Restaurant',
 'Steakhouse',
 'Sandwich Place',
 'Deli / Bodega',
 'Chinese Restaurant',
 'Cocktail Bar',
 'Greek Restaurant',
 'English Restaurant',
 'Noodle House',
 'American Restaurant',
 'Bubble Tea Shop',
 'Bar',
 'New American Restaurant',
 'Malay Restaurant',
 'Vietnamese Restaurant',
 'Hotpot Restaurant',
 'Dim Sum Restaurant',
 'Italian Restaurant',
 'Spanish Restaurant',
 'Asian Restaurant',
 'Snack Place',
 'Restaurant',
 'Dumpling Restaurant',
 'Bakery',
 'Taiwanese Restaurant',
 'Dessert Shop',
 'Shanghai Restaurant',
 'Austrian Restaurant',
 'Mexican Restaurant',
 'Ramen Restaurant',
 'Café',
 'Burger Joint',
 'Tapas Restaurant',
 'Breakfast Spot',
 'Wine Shop',
 'Caribbean Restaurant',
 'Indian Restaurant',
 'Latin American Restaurant',
 'Sushi Restaurant',
 'Wine Bar',
 'Beer Bar',
 'Arepa Restaurant',
 'Bistro',
 'Fast Food Restaurant',
 'Juice Bar',
 'Japanese Restaurant',
 'Mediterranean Restaurant',
 'Pub',
 'Gastropub',
 'BBQ Joint',
 'Japanese Curry Restaurant',
 'Beer Garden',
 'Falafel Restaurant',
 'Food & Drink Shop',
 'Cuban Restaurant',
 'Food Court',
 'French Restaurant',
 'Ethiopian Restaurant',
 'African Restaurant',
 'Bagel Shop',
 'Fried Chicken Joint',
 'Cafeteria',
 'Southern / Soul Food Restaurant',
 'Thai Restaurant',
 'Taco Place',
 'Vegetarian / Vegan Restaurant',
 'Burrito Place',
 'Hotel Bar',
 'Salad Place',
 'Turkish Restaurant',
 'German Restaurant',
 'Hot Dog Joint',
 'Peruvian Restaurant',
 'Sports Bar',
 'Afghan Restaurant',
 'Czech Restaurant',
 'Middle Eastern Restaurant',
 'Australian Restaurant',
 'South Indian Restaurant',
 'Caucasian Restaurant',
 'Szechuan Restaurant',
 'Hawaiian Restaurant',
 'Brazilian Restaurant',
 'Jewish Restaurant',
 'Scandinavian Restaurant',
 'Paella Restaurant',
 'Israeli Restaurant',
 'Udon Restaurant',
 'Eastern European Restaurant',
 'Gaming Cafe',
 'Lebanese Restaurant',
 'Moroccan Restaurant',
 'Korean Restaurant',
 'Pet Café',
 'Swiss Restaurant',
 'Soup Place',
 'Filipino Restaurant',
 'Argentinian Restaurant',
 'Whisky Bar',
 'Modern European Restaurant',
 'Karaoke Bar',
 'Wings Joint',
 'Irish Pub',
 'Belgian Restaurant',
 'Tiki Bar',
 'Kosher Restaurant',
 'Tibetan Restaurant',
 'Sake Bar',
 'Venezuelan Restaurant',
 'Molecular Gastronomy Restaurant',
 'Street Food Gathering',
 'Persian Restaurant',
 'Cambodian Restaurant',
 'Soba Restaurant',
 'Russian Restaurant',
 'Portuguese Restaurant',
 'Pakistani Restaurant',
 'Organic Grocery',
 'Grocery Store',
 'Market',
 'Frozen Yogurt Shop',
 'Supermarket',
 'Farmers Market',
 'Convenience Store',
 'Health Food Store',
 'Fish Market',
 'Flea Market']

And let's examine the new dataframe size.

In [32]:
manhattan_onehot.shape

(3289, 330)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant

#### Let's confirm the new size

In [34]:
manhattan_grouped.shape

(40, 330)

#### Let's print each neighborhood along with the top 5 most common venues

In [35]:
manhattan_grouped = manhattan_onehot[needed_features].groupby('Neighborhood').sum().reset_index()
manhattan_grouped.head()

,Neighborhood,Pizza Place,Diner,Coffee Shop,Donut Shop,Seafood Restaurant,Comfort Food Restaurant,Steakhouse,Sandwich Place,Deli / Bodega,Chinese Restaurant,Cocktail Bar,Greek Restaurant,English Restaurant,Noodle House,American Restaurant,Bubble Tea Shop,Bar,New American Restaurant,Malay Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Dim Sum Restaurant,Italian Restaurant,Spanish Restaurant,Asian Restaurant,Snack Place,Restaurant,Dumpling Restaurant,Bakery,Taiwanese Restaurant,Dessert Shop,Shanghai Restaurant,Austrian Restaurant,Mexican Restaurant,Ramen Restaurant,Café,Burger Joint,Tapas Restaurant,Breakfast Spot,Wine Shop,Caribbean Restaurant,Indian Restaurant,Latin American Restaurant,Sushi Restaurant,Wine Bar,Beer Bar,Arepa Restaurant,Bistro,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Mediterranean Restaurant,Pub,Gastropub,BBQ Joint,Japanese Curry Restaurant,Beer Garden,Falafel Restaurant,Food & Drink Shop,Cuban Restaurant,Food Court,French Restaurant,Ethiopian Restaurant,African Restaurant,Bagel Shop,Fried Chicken Joint,Cafeteria,Southern / Soul Food Restaurant,Thai Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Burrito Place,Hotel Bar,Salad Place,Turkish Restaurant,German Restaurant,Hot Dog Joint,Peruvian Restaurant,Sports Bar,Afghan Restaurant,Czech Restaurant,Middle Eastern Restaurant,Australian Restaurant,South Indian Restaurant,Caucasian Restaurant,Szechuan Restaurant,Hawaiian Restaurant,Brazilian Restaurant,Jewish Restaurant,Scandinavian Restaurant,Paella Restaurant,Israeli Restaurant,Udon Restaurant,Eastern European Restaurant,Gaming Cafe,Lebanese Restaurant,Moroccan Restaurant,Korean Restaurant,Pet Café,Swiss Restaurant,Soup Place,Filipino Restaurant,Argentinian Restaurant,Whisky Bar,Modern European Restaurant,Karaoke Bar,Wings Joint,Irish Pub,Belgian Restaurant,Tiki Bar,Kosher Restaurant,Tibetan Restaurant,Sake Bar,Venezuelan Restaurant,Molecular Gastronomy Restaurant,Street Food Gathering,Persian Restaurant,Cambodian Restaurant,Soba Restaurant,Russian Restaurant,Portuguese Restaurant,Pakistani Restaurant,Organic Grocery,Grocery Store,Market,Frozen Yogurt Shop,Supermarket,Farmers Market,Convenience Store,Health Food Store,Fish Market,Flea Market
0,Battery Park City,2,0,8,0,0,0,1,2,0,1,0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,0,0,0,1,0,0,2,0,0,3,0,1,0,1,1,0,0,1,0,1,0,0,1,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0
1,Carnegie Hill,6,0,5,0,0,0,0,0,1,1,2,0,0,0,1,0,3,1,0,1,0,0,2,0,0,0,1,0,2,0,0,0,0,1,1,4,1,0,1,3,0,1,0,1,1,0,0,0,0,0,3,1,2,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0
2,Central Harlem,1,0,0,0,2,0,0,0,0,2,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,2,1,3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,Chelsea,1,0,6,0,3,0,1,1,0,0,1,0,0,1,3,0,1,1,0,0,0,0,6,0,2,0,0,0,3,0,1,0,0,1,0,2,1,2,1,1,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1
4,Chinatown,1,0,2,0,0,0,0,2,0,9,5,1,1,3,4,3,2,1,2,4,3,4,1,1,2,1,1,3,3,1,1,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0


In [36]:


manhattan_grouped['Total Restaurants'] = manhattan_grouped[restaurant_list].sum(axis = 1)
manhattan_grouped['Total Grocery Store'] = manhattan_grouped[grocery_list].sum(axis = 1)
manhattan_grouped



,Neighborhood,Pizza Place,Diner,Coffee Shop,Donut Shop,Seafood Restaurant,Comfort Food Restaurant,Steakhouse,Sandwich Place,Deli / Bodega,Chinese Restaurant,Cocktail Bar,Greek Restaurant,English Restaurant,Noodle House,American Restaurant,Bubble Tea Shop,Bar,New American Restaurant,Malay Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Dim Sum Restaurant,Italian Restaurant,Spanish Restaurant,Asian Restaurant,Snack Place,Restaurant,Dumpling Restaurant,Bakery,Taiwanese Restaurant,Dessert Shop,Shanghai Restaurant,Austrian Restaurant,Mexican Restaurant,Ramen Restaurant,Café,Burger Joint,Tapas Restaurant,Breakfast Spot,Wine Shop,Caribbean Restaurant,Indian Restaurant,Latin American Restaurant,Sushi Restaurant,Wine Bar,Beer Bar,Arepa Restaurant,Bistro,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Mediterranean Restaurant,Pub,Gastropub,BBQ Joint,Japanese Curry Restaurant,Beer Garden,Falafel Restaurant,Food & Drink Shop,Cuban Restaurant,Food Court,French Restaurant,Ethiopian Restaurant,African Restaurant,Bagel Shop,Fried Chicken Joint,Cafeteria,Southern / Soul Food Restaurant,Thai Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Burrito Place,Hotel Bar,Salad Place,Turkish Restaurant,German Restaurant,Hot Dog Joint,Peruvian Restaurant,Sports Bar,Afghan Restaurant,Czech Restaurant,Middle Eastern Restaurant,Australian Restaurant,South Indian Restaurant,Caucasian Restaurant,Szechuan Restaurant,Hawaiian Restaurant,Brazilian Restaurant,Jewish Restaurant,Scandinavian Restaurant,Paella Restaurant,Israeli Restaurant,Udon Restaurant,Eastern European Restaurant,Gaming Cafe,Lebanese Restaurant,Moroccan Restaurant,Korean Restaurant,Pet Café,Swiss Restaurant,Soup Place,Filipino Restaurant,Argentinian Restaurant,Whisky Bar,Modern European Restaurant,Karaoke Bar,Wings Joint,Irish Pub,Belgian Restaurant,Tiki Bar,Kosher Restaurant,Tibetan Restaurant,Sake Bar,Venezuelan Restaurant,Molecular Gastronomy Restaurant,Street Food Gathering,Persian Restaurant,Cambodian Restaurant,Soba Restaurant,Russian Restaurant,Portuguese Restaurant,Pakistani Restaurant,Organic Grocery,Grocery Store,Market,Frozen Yogurt Shop,Supermarket,Farmers Market,Convenience Store,Health Food Store,Fish Market,Flea Market,Total Restaurants,Total Grocery Store
0,Battery Park City,2,0,8,0,0,0,1,2,0,1,0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,0,0,0,1,0,0,2,0,0,3,0,1,0,1,1,0,0,1,0,1,0,0,1,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,39,3
1,Carnegie Hill,6,0,5,0,0,0,0,0,1,1,2,0,0,0,1,0,3,1,0,1,0,0,2,0,0,0,1,0,2,0,0,0,0,1,1,4,1,0,1,3,0,1,0,1,1,0,0,0,0,0,3,1,2,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,59,4
2,Central Harlem,1,0,0,0,2,0,0,0,0,2,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,2,1,3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,25,1
3,Chelsea,1,0,6,0,3,0,1,1,0,0,1,0,0,1,3,0,1,1,0,0,0,0,6,0,2,0,0,0,3,0,1,0,0,1,0,2,1,2,1,1,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,50,4
4,Chinatown,1,0,2,0,0,0,0,2,0,9,5,1,1,3,4,3,2,1,2,4,3,4,1,1,2,1,1,3,3,1,1,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,65,2
5,Civic Center,0,1,3,0,0,0,0,3,0,0,3,0,0,0,2,1,1,1,0,0,0,0,4,0,1,0,0,0,5,0,0,0,0,1,1,1,0,0,0,2,0,1,0,3,1,0,0,0,0,0,1,0,0,0,0,0,0,2,0,1,0,3,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,49,0
6,Clinton,1,1,4,0,2,0,1,2,1,1,1,0,0,1,4,0,1,2,

In [37]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(manhattan_grouped_clustering)
kmeans.labels_

array([2, 0, 2, 3, 4, 3, 3, 2, 4, 0, 3, 0, 4, 3, 2, 2, 0, 2, 4, 2, 3, 2,
       1, 3, 0, 2, 4, 0, 1, 2, 1, 0, 3, 3, 4, 3, 4, 3, 0, 4], dtype=int32)

In [38]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = manhattan_grouped_clustering.columns
means_df.index = ['C0','C1','C2','C3','C4']
means_df

,Pizza Place,Diner,Coffee Shop,Donut Shop,Seafood Restaurant,Comfort Food Restaurant,Steakhouse,Sandwich Place,Deli / Bodega,Chinese Restaurant,Cocktail Bar,Greek Restaurant,English Restaurant,Noodle House,American Restaurant,Bubble Tea Shop,Bar,New American Restaurant,Malay Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Dim Sum Restaurant,Italian Restaurant,Spanish Restaurant,Asian Restaurant,Snack Place,Restaurant,Dumpling Restaurant,Bakery,Taiwanese Restaurant,Dessert Shop,Shanghai Restaurant,Austrian Restaurant,Mexican Restaurant,Ramen Restaurant,Café,Burger Joint,Tapas Restaurant,Breakfast Spot,Wine Shop,Caribbean Restaurant,Indian Restaurant,Latin American Restaurant,Sushi Restaurant,Wine Bar,Beer Bar,Arepa Restaurant,Bistro,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Mediterranean Restaurant,Pub,Gastropub,BBQ Joint,Japanese Curry Restaurant,Beer Garden,Falafel Restaurant,Food & Drink Shop,Cuban Restaurant,Food Court,French Restaurant,Ethiopian Restaurant,African Restaurant,Bagel Shop,Fried Chicken Joint,Cafeteria,Southern / Soul Food Restaurant,Thai Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Burrito Place,Hotel Bar,Salad Place,Turkish Restaurant,German Restaurant,Hot Dog Joint,Peruvian Restaurant,Sports Bar,Afghan Restaurant,Czech Restaurant,Middle Eastern Restaurant,Australian Restaurant,South Indian Restaurant,Caucasian Restaurant,Szechuan Restaurant,Hawaiian Restaurant,Brazilian Restaurant,Jewish Restaurant,Scandinavian Restaurant,Paella Restaurant,Israeli Restaurant,Udon Restaurant,Eastern European Restaurant,Gaming Cafe,Lebanese Restaurant,Moroccan Restaurant,Korean Restaurant,Pet Café,Swiss Restaurant,Soup Place,Filipino Restaurant,Argentinian Restaurant,Whisky Bar,Modern European Restaurant,Karaoke Bar,Wings Joint,Irish Pub,Belgian Restaurant,Tiki Bar,Kosher Restaurant,Tibetan Restaurant,Sake Bar,Venezuelan Restaurant,Molecular Gastronomy Restaurant,Street Food Gathering,Persian Restaurant,Cambodian Restaurant,Soba Restaurant,Russian Restaurant,Portuguese Restaurant,Pakistani Restaurant,Organic Grocery,Grocery Store,Market,Frozen Yogurt Shop,Supermarket,Farmers Market,Convenience Store,Health Food Store,Fish Market,Flea Market,Total Restaurants,Total Grocery Store
C0,3.250000,0.500000,4.500000,-2.775558e-17,0.625000,3.469447e-18,1.250000,0.750000,0.875000,0.875000,2.125000,0.375000,3.469447e-18,-2.775558e-17,2.250000,0.125000,1.500000,1.125000,1.387779e-17,5.000000e-01,0.000000e+00,0.000000,5.375000e+00,0.125000,0.125000,0.125000,0.875000,0.0,1.625000,6.938894e-18,0.750000,0.000000e+00,1.250000e-01,1.750000,0.250000,1.250000,1.000000,0.125000,2.500000e-01,1.875000,5.551115e-17,0.875,3.750000e-01,2.000000,1.250000,3.750000e-01,0.000000e+00,0.000000e+00,0.000000,0.875000,1.750000,0.500000,0.625000,0.750000,1.250000e-01,1.250000e-01,2.500000e-01,3.750000e-01,1.250000e-01,0.250000,0.375000,1.625000e+00,0.125000,0.000000e+00,1.000000,2.500000e-01,3.469447e-18,1.250000e-01,0.875000,0.625000,8.750000e-01,6.938894e-18,6.250000e-01,0.375000,2.500000e-01,0.125000,1.250000e-01,0.000000,0.250000,1.250000e-01,1.250000e-01,2.500000e-01,0.000000e+00,3.469447e-18,6.938894e-18,1.250000e-01,1.387779e-17,1.250000e-01,0.000000e+00,3.469447e-18,6.938894e-18,1.250000e-01,3.469447e-18,3.469447e-18,3.469447e-18,1.250000e-01,1.250000e-01,2.125000e+00,6.938894e-18,0.000000e+00,3.469447e-18,0.25,1.250000e-01,1.250000e-01,3.469447e-18,2.500000e-01,1.250000e-01,1.250000e-01,1.250000e-01,1.250000e-01,1.250000e-01,1.250000e-01,1.250000e-01,1.250000e-01,3.469447e-18,1.250000e-01,1.250000e-01,1.250000e-01,3.469447e-18,3.469447e-18,3.469447e-18,3.469447e-18,6.938894e-18,1.625000e+00,2.500000e-01,0.125000,0.375000,0.125000,0.125000,0.125000,3.469447e-18,6.938894e-18,59.875000,2.750000
C1,0.666667,0.333333,1.333333,3.333333e-01,0.333333,3.333333e-01,0.333333,1.000000,1.000000,0.000000,0.333333,0.333333,-3.469447e-18,0.000000e+00,0.000000,0.000000,1.333333,0.000000,-1.387779e-17,5.551115e-17,0.000000e+00,0.000000,-4.440892e-16,0.

In [39]:
col_list = list(manhattan_grouped_clustering)
col_list.remove('Total Grocery Store')
means_df['Total Sum'] = means_df[col_list].sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Pizza Place,Diner,Coffee Shop,Donut Shop,Seafood Restaurant,Comfort Food Restaurant,Steakhouse,Sandwich Place,Deli / Bodega,Chinese Restaurant,Cocktail Bar,Greek Restaurant,English Restaurant,Noodle House,American Restaurant,Bubble Tea Shop,Bar,New American Restaurant,Malay Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Dim Sum Restaurant,Italian Restaurant,Spanish Restaurant,Asian Restaurant,Snack Place,Restaurant,Dumpling Restaurant,Bakery,Taiwanese Restaurant,Dessert Shop,Shanghai Restaurant,Austrian Restaurant,Mexican Restaurant,Ramen Restaurant,Café,Burger Joint,Tapas Restaurant,Breakfast Spot,Wine Shop,Caribbean Restaurant,Indian Restaurant,Latin American Restaurant,Sushi Restaurant,Wine Bar,Beer Bar,Arepa Restaurant,Bistro,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Mediterranean Restaurant,Pub,Gastropub,BBQ Joint,Japanese Curry Restaurant,Beer Garden,Falafel Restaurant,Food & Drink Shop,Cuban Restaurant,Food Court,French Restaurant,Ethiopian Restaurant,African Restaurant,Bagel Shop,Fried Chicken Joint,Cafeteria,Southern / Soul Food Restaurant,Thai Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Burrito Place,Hotel Bar,Salad Place,Turkish Restaurant,German Restaurant,Hot Dog Joint,Peruvian Restaurant,Sports Bar,Afghan Restaurant,Czech Restaurant,Middle Eastern Restaurant,Australian Restaurant,South Indian Restaurant,Caucasian Restaurant,Szechuan Restaurant,Hawaiian Restaurant,Brazilian Restaurant,Jewish Restaurant,Scandinavian Restaurant,Paella Restaurant,Israeli Restaurant,Udon Restaurant,Eastern European Restaurant,Gaming Cafe,Lebanese Restaurant,Moroccan Restaurant,Korean Restaurant,Pet Café,Swiss Restaurant,Soup Place,Filipino Restaurant,Argentinian Restaurant,Whisky Bar,Modern European Restaurant,Karaoke Bar,Wings Joint,Irish Pub,Belgian Restaurant,Tiki Bar,Kosher Restaurant,Tibetan Restaurant,Sake Bar,Venezuelan Restaurant,Molecular Gastronomy Restaurant,Street Food Gathering,Persian Restaurant,Cambodian Restaurant,Soba Restaurant,Russian Restaurant,Portuguese Restaurant,Pakistani Restaurant,Organic Grocery,Grocery Store,Market,Frozen Yogurt Shop,Supermarket,Farmers Market,Convenience Store,Health Food Store,Fish Market,Flea Market,Total Restaurants,Total Grocery Store,Total Sum
C4,2.000000,0.625000,3.125000,1.250000e-01,1.625000,3.469447e-18,0.875000,1.625000,0.750000,2.750000,2.000000,0.500000,1.250000e-01,8.750000e-01,2.000000,0.875000,3.000000,0.750000,3.750000e-01,1.375000e+00,3.750000e-01,0.500000,4.375000e+00,0.625000,0.750000,0.375000,0.625000,0.5,2.625000,2.500000e-01,1.000000,2.500000e-01,1.250000e-01,1.750000,0.625000,1.750000,1.125000,0.250000,3.750000e-01,0.750000,3.750000e-01,1.250,2.500000e-01,1.875000,2.250000,1.250000e-01,2.500000e-01,1.250000e-01,0.000000,0.500000,1.750000,1.000000,0.875000,0.125000,1.250000e-01,1.250000e-01,1.387779e-17,1.250000e-01,1.250000e-01,0.375000,0.000000,1.875000e+00,0.000000,0.000000e+00,1.250000,1.250000e-01,3.469447e-18,1.250000e-01,1.000000,0.250000,1.375000e+00,6.938894e-18,5.551115e-17,0.250000,3.750000e-01,0.250000,2.500000e-01,0.375000,0.250000,3.469447e-18,3.469447e-18,5.000000e-01,2.500000e-01,1.250000e-01,6.938894e-18,1.387779e-17,1.250000e-01,6.938894e-18,2.500000e-01,1.250000e-01,1.250000e-01,1.250000e-01,1.250000e-01,1.250000e-01,1.250000e-01,1.250000e-01,1.250000e-01,2.500000e-01,1.250000e-01,2.500000e-01,1.250000e-01,0.25,1.250000e-01,0.000000e+00,3.469447e-18,3.750000e-01,0.000000e+00,3.469447e-18,3.469447e-18,3.469447e-18,3.469447e-18,3.469447e-18,1.250000e-01,3.469447e-18,3.469447e-18,3.469447e-18,3.469447e-18,3.469447e-18,1.250000e-01,3.469447e-18,3.469447e-18,3.469447e-18,2.500000e-01,3.750000e-01,2.775558e-17,0.000000,0.125000,0.375000,0.000000,0.125000,3.469447e-18,1.250000e-01,67.750000,1.375000,136.875000
C0,3.250000,0.500000,4.500000,-2.775558e-17,0.625000,3.469447e-18,1.250000,0.750000,0.875000,0.875000,2.125000,0.375000,3.469447e-18,-2.775558e-17,2.250000,0.125000,1.500000,1.125000,1.387779e-17,5.000000e-01,0.000000e+00,0.0000


## Suggestion for the top 3 Manhattan Neighborhood Clusters to open a new Grocery store

Best Cluster is C3

Second Best Cluster is C4

Third Best Cluster is C0


In [40]:
manhattan_grouped1 = manhattan_grouped.set_index('Neighborhood')
manhattan_cluster = pd.DataFrame([manhattan_grouped1.index, kmeans.labels_]).T
manhattan_cluster.columns = ['Neighborhood', 'Cluster Labels']
manhattan_cluster.head()

,Neighborhood,Cluster Labels
0,Battery Park City,2
1,Carnegie Hill,0
2,Central Harlem,2
3,Chelsea,3
4,Chinatown,4



## Neighbourhoods in the Top 3 Clusters¶

Cluster 3


In [41]:
manhattan_cluster[manhattan_cluster['Cluster Labels'] == 3]

,Neighborhood,Cluster Labels
3,Chelsea,3
5,Civic Center,3
6,Clinton,3
10,Flatiron,3
13,Hamilton Heights,3
20,Manhattan Valley,3
23,Midtown,3
32,Tribeca,3
33,Tudor City,3
35,Upper East Side,3


Cluster 4

In [42]:
manhattan_cluster[manhattan_cluster['Cluster Labels'] == 4]

,Neighborhood,Cluster Labels
4,Chinatown,4
8,East Village,4
12,Greenwich Village,4
18,Little Italy,4
26,Murray Hill,4
34,Turtle Bay,4
36,Upper West Side,4
39,Yorkville,4


Cluster 0

In [43]:
manhattan_cluster[manhattan_cluster['Cluster Labels'] == 0]

,Neighborhood,Cluster Labels
1,Carnegie Hill,0
9,Financial District,0
11,Gramercy,0
16,Lenox Hill,0
24,Midtown South,0
27,Noho,0
31,Sutton Place,0
38,West Village,0


## Clusters on the Manhattan map.
The markdowns colored in Red, Purple and Blue are the Top 3 neighborhood clusters where a new grocery store is suggested to be opened.

In [44]:
manhattan_merged = manhattan_data
manhattan_merged['Cluster Labels'] = kmeans.labels_
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,Manhattan,Marble Hill,40.876551,-73.910660,2
1,Manhattan,Chinatown,40.715618,-73.994279,0
2,Manhattan,Washington Heights,40.851903,-73.936900,2
3,Manhattan,Inwood,40.867684,-73.921210,3
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4


In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

kclusters = 5

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters